<a href="https://colab.research.google.com/github/financieras/big_data/blob/main/leccion_2_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lección 2.3.5: Integración de BI con pipelines de datos

## 1. ¿Por qué integrar BI con pipelines de datos?

Los datos son como el combustible de un coche: sin un suministro constante, limpio y confiable, incluso el mejor dashboard de Business Intelligence (BI) se queda parado. La **integración de BI con pipelines de datos** asegura que los datos lleguen frescos, estructurados y listos para ser analizados, transformando reportes estáticos en **herramientas dinámicas para decisiones en tiempo real**.

**Beneficios clave:**
- **Actualización automática:** Dashboards siempre reflejan la realidad.
- **Consistencia:** Evita métricas contradictorias entre equipos.
- **Escalabilidad:** Soporta desde startups hasta petabytes de datos.
- **Eficiencia:** Reduce el tiempo manual de analistas.

> **Analogía:** Un pipeline es como la cocina de un restaurante: prepara los ingredientes (datos) para que el chef (BI) cree platos deliciosos (dashboards).

**Problema sin integración:**  
Datos obsoletos, procesos manuales y errores humanos. Ejemplo: un retailer que actualiza su inventario semanalmente pierde ventas por falta de stock.

---

## 2. Recordatorio: Pipelines y BI

- **Pipeline de datos:** Secuencia automatizada que **extrae** datos (de APIs, bases de datos), los **transforma** (limpieza, agregación) y los **carga** en un destino (data warehouse). Puede ser **ETL** (transformación antes de cargar) o **ELT** (transformación en la nube).
- **BI (Business Intelligence):** Herramientas como Power BI, Tableau o Looker Studio que visualizan datos para responder preguntas de negocio. Sin pipelines, dependen de procesos manuales.

> **Ejemplo:** En una app de delivery, un pipeline extrae datos de pedidos en tiempo real, calcula métricas (tiempo promedio de entrega) y las carga en BigQuery. El dashboard de Looker Studio muestra estas métricas al equipo de operaciones cada 10 minutos.

---

## 3. Arquitectura de integración: Cómo conectar los puntos

```mermaid
graph LR
    A[Fuentes: CRM, APIs, Bases de datos] --> B(Pipeline ETL/ELT)
    B --> |Transformación| C[Data Warehouse: BigQuery, Snowflake]
    C --> D[BI: Power BI, Tableau, Looker Studio]
    D --> E[Usuarios: Analistas, Gerentes, CEOs]
```

**Componentes clave:**
1. **Fuentes:** APIs (Salesforce), bases de datos (PostgreSQL), archivos (CSV, JSON).
2. **Pipeline:** Herramientas como Airflow (orquestación), dbt (transformación SQL) o Spark (procesamiento masivo).
3. **Almacenamiento:** Data warehouses (Snowflake, BigQuery) o data lakes (AWS S3).
4. **BI:** Conexión directa via ODBC/JDBC o APIs nativas.

> **Tip:** Usa un **data warehouse** como punto central para garantizar datos limpios y optimizados para BI.

---

## 4. Ejemplo práctico: Retail en tiempo real

**Contexto:**  
"ModaRápida", una cadena de 30 tiendas, necesita un dashboard para monitorear ventas e inventario en tiempo real.

**Pipeline implementado:**
```
[Ventas en tiendas] → [API REST] → [Kafka para streaming] → [Spark para transformación] → [BigQuery] → [Power BI]
[Inventario RFID] → [API inventario] →
```

**Código ilustrativo (Python + Spark):**
```python
from pyspark.sql import SparkSession

# Iniciar Spark
spark = SparkSession.builder.appName("RetailPipeline").getOrCreate()

# Extraer datos de Kafka
ventas = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "ventas").load()

# Transformar: calcular ventas por región
ventas_por_region = ventas.groupBy("region").agg({"monto": "sum"})

# Cargar en BigQuery
ventas_por_region.writeStream.format("bigquery").option("table", "modarapida.ventas").start()
```

**Dashboard en Power BI:**
- Gráfico de barras: Ventas por región.
- Tabla: Stock crítico por tienda.
- Actualización: Cada 15 minutos.

**Impacto:**
| Métrica | Antes | Después |
|---------|-------|---------|
| Tiempo de actualización | 1 día | 15 minutos |
| Ventas perdidas por stock | 10% | 2% |
| Decisiones diarias | 3 | 12 |

**Resultado:** ModaRápida ajusta inventario en tiempo real, aumentando ventas un 15%.

---

## 5. Herramientas para la integración

| Herramienta | Rol | Conector BI | Ventaja | Caso de uso |
|-------------|-----|-------------|---------|-------------|
| **Apache Airflow** | Orquestación | Indirecto (via warehouse) | Programación flexible | Automatizar ETL nocturno |
| **dbt** | Transformación | SQL a BI | Modelado sencillo | Estandarizar métricas |
| **Snowflake** | Data Warehouse | ODBC/JDBC | Escalabilidad cloud | Análisis masivo |
| **Power BI** | BI | Nativo a warehouses | Integración Microsoft | Reportes corporativos |
| **Looker Studio** | BI | Google Cloud | Gratuito, fácil | Startups y PYMEs |

> **Recomendación:** Para empezar, prueba **dbt + BigQuery + Looker Studio**. Es accesible y potente.

---

## 6. Comparación: Batch vs. Streaming

| Aspecto | Batch (Lotes) | Streaming (Tiempo real) |
|---------|---------------|-------------------------|
| **Frecuencia** | Diaria, horaria | Segundos, minutos |
| **Ejemplo** | Reporte de ventas nocturno | Monitoreo de fraudes |
| **Herramientas** | Airflow, dbt | Kafka, Spark Streaming |
| **Ventaja** | Simple, económico | Reacción inmediata |
| **Desafío** | Retraso en datos | Complejidad técnica |

**Ejemplo de código (Batch con Airflow):**
```python
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

def extract_sales():
    # Extraer datos
    pass

def transform_sales():
    # Transformar con Pandas/dbt
    pass

def load_to_bigquery():
    # Cargar a BI
    pass

dag = DAG('ventas_diarias', start_date=datetime(2025, 1, 1), schedule_interval='@daily')
extract = PythonOperator(task_id='extract', python_callable=extract_sales, dag=dag)
transform = PythonOperator(task_id='transform', python_callable=transform_sales, dag=dag)
load = PythonOperator(task_id='load', python_callable=load_to_bigquery, dag=dag)

extract >> transform >> load
```

---

## 7. Mejores prácticas

1. **Colabora entre equipos:** Ingenieros y analistas deben definir métricas juntos.
2. **Centraliza transformaciones:** Usa dbt o SQL para evitar lógica duplicada.
3. **Monitorea fallos:** Configura alertas con herramientas como Datadog o Prefect.
4. **Documenta:** Usa catálogos (DataHub) para rastrear el linaje de datos.
5. **Optimiza performance:** Precalcula agregaciones en el warehouse para acelerar BI.

> **Tip práctico:** Empieza con un pipeline pequeño (CSV → Google Sheets → Looker Studio) antes de escalar a stacks complejos.

---

## 8. Errores comunes y soluciones

| Error | Impacto | Solución |
|-------|---------|----------|
| Pipeline sin monitoreo | Dashboards obsoletos | Alertas automáticas |
| Métricas inconsistentes | Decisiones erróneas | Catálogo de datos centralizado |
| Conexión directa a fuentes | BI lento, inestable | Usar data warehouse |
| Falta de pruebas | Datos erróneos | Validaciones con Great Expectations |

---

## 9. Tendencias emergentes

- **Reverse ETL:** Llevar datos de BI a sistemas operativos (ej. enviar alertas a Slack con Census).
- **Headless BI:** APIs como Cube.js para integrar BI sin interfaces gráficas.
- **Augmented Analytics:** Herramientas como ThoughtSpot que usan IA para generar insights automáticamente.

---

## 10. Actividad para alumnos

**Mini-proyecto:**
1. Descarga un dataset público (ej. ventas en Kaggle).
2. Crea un pipeline simple con Python/Pandas que limpie y agregue datos.
3. Carga el resultado en Google Sheets o BigQuery (free tier).
4. Conecta a Looker Studio y crea un dashboard con 2 visualizaciones.
5. Reflexiona: ¿Qué pasa si cambias la estructura de los datos? ¿Cómo lo detectas?

---

## 11. Resumen

- **Pipelines:** Automatizan el flujo de datos desde fuentes hasta destinos.
- **BI:** Visualiza datos para decisiones rápidas.
- **Integración:** Conecta ambos para dashboards frescos y confiables.
- **Herramientas clave:** Airflow, dbt, BigQuery, Power BI.
- **Impacto:** Reduce el tiempo entre datos y decisiones, impulsando el negocio.

> **Frase inspiradora:** “Un dashboard sin pipeline es como un cuadro sin pintura: solo un marco vacío.”

---

## 12. Referencias

### Vídeos
- [Building Modern Data Pipelines for BI](https://youtu.be/Jw_0Zrz0YqM)
- [Power BI and Azure Data Factory Integration](https://youtu.be/0pK_6zL2YfU)
- [dbt + BI: A Match Made in Heaven](https://youtu.be/abc123)

### Lecturas
- [Data Pipelines Pocket Reference](https://www.oreilly.com/library/view/data-pipelines-pocket/9781492087823/)
- [Modern Data Stack Guide](https://www.moderndatastack.xyz/)
- [dbt Best Practices](https://docs.getdbt.com/guides/best-practices)